# Целевой признак

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

Загружаем данные о времени и статусах сабмитов к практическим заданиям за период с 15.06.2015г. по 19.05.2018г.

In [3]:
submissions_data = pd.read_csv('DATA\submissions_data_train.zip', compression='zip')

Составим сводную таблицу

In [4]:
users_scores = submissions_data.pivot_table(index='user_id',
                                            columns='submission_status',
                                            values='step_id',
                                            aggfunc='nunique',
                                            fill_value=0).reset_index()

In [5]:
#Избавимся от полученного нерелевантного названия индекса submission_status
users_scores.columns
users_scores.columns.name
users_scores.columns.name = None

Загружаем данные о действиях, которые совершают студенты со стэпами

In [6]:
events_data = pd.read_csv('DATA\event_data_train.zip', compression='zip')

Составим сводную таблицу

In [7]:
users_actions = events_data.pivot_table(index='user_id',
                                        columns='action',
                                        values='step_id',
                                        aggfunc='nunique',
                                        fill_value=0).reset_index()

In [8]:
#Избавимся от полученного нерелевантного названия индекса action
users_actions.columns
users_actions.columns.name
users_actions.columns.name = None

Проведем слияние users_actions и users_scores

In [9]:
users_actions = users_actions.merge(users_scores,on='user_id',how='outer')

In [10]:
#Пропуски заменим на ноль
users_actions = users_actions.fillna(0)

Создадим и заполним колонку, соответствующую признаку успешного окончания курса

In [11]:
users_actions['is_gone'] = users_actions.correct > 40

Создадим целевую переменную

In [12]:
y = users_actions[['user_id', 'is_gone']]

In [13]:
y['is_gone'] = y['is_gone'].map(int)

In [14]:
print ('Распределение таргета')
pd.cut(y.is_gone, 2, labels=["0", "1"]).value_counts()

Распределение таргета


0    17310
1     1924
Name: is_gone, dtype: int64

Количество пользователей не завершивших курс значительно превышает тех кто успешно его закончил, создавая дисбаланс классов

Сохраним в файл

In [15]:
y.to_csv('DATA\\target_feature.csv', index=False)